<a href="https://colab.research.google.com/github/weiyuchen1213/Active-Learning/blob/master/Active_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>